In [6]:
from net.net import model
from utils.dataset import FaceKeyPointsDatasets
import tensorflow as tf
import numpy as np

In [7]:
net = model
net.load_weights("weights/face_240_240_50.h5")

In [8]:
def representative_data_gen():
  for i in range(31):
    data = FaceKeyPointsDatasets(32,'train')[0][0][31]
    img = np.expand_dims(data,0)
    img = np.array(img,dtype=np.float32)  
    yield [img]

In [9]:
# 量化模型
converter = tf.lite.TFLiteConverter.from_keras_model(model=net)
converter._experimental_disable_per_channel = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_quantized_model = converter.convert()
# 查看模型大小
quantized_model_size = len(tflite_quantized_model) / (1024*1024)
print(f'Quantized model size = {quantized_model_size}MBs')

INFO:tensorflow:Assets written to: C:\Users\27260\AppData\Local\Temp\tmp4gy85zrf\assets


INFO:tensorflow:Assets written to: C:\Users\27260\AppData\Local\Temp\tmp4gy85zrf\assets
d:\Configurations\Miniconda\envs\Tensor\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Quantized model size = 3.5501632690429688MBs


In [10]:
f = open('face_240_240_uint8_50.tflite', "wb")
f.write(tflite_quantized_model)
f.close()